1 - Create a repo in Github and clone it on local drive
2 - Set up Python environnement
3 - Create core folder to keep my functions
4 - Create data folder
5 - function summarise
6 - check for quick linear regression
 python3 -m ipykernel install --user --name venv


# Packages

In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

# Personal Packages

In [2]:
from core.helpers import summarize
from core.helpers import transform_df_for_model
from core.helpers import sklearn_sm

# Linear regression : 1 variable
## Simple way

In [3]:
Boston = pd.read_csv("data/Boston.csv")
df = pd.DataFrame({'intercept': np.ones(Boston.shape[0]),
                  'lstat': Boston['lstat']})
y = Boston['medv']
model = sm.OLS(y, df)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,34.5538,0.563,61.415,0.0
lstat,-0.9500,0.039,-24.528,0.0


## Using my own function

In [4]:
X = transform_df_for_model(df,['lstat'])
y = Boston['medv']
model = sm.OLS(y, X)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,34.5538,0.563,61.415,0.0
lstat,-0.9500,0.039,-24.528,0.0


In [5]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     601.6
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           5.08e-88
Time:                        21:49:56   Log-Likelihood:                -1641.5
No. Observations:                 506   AIC:                             3287.
Df Residuals:                     504   BIC:                             3295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     34.5538      0.563     61.415      0.000      33.448      35.659
lstat         -0.9500      0.039    -24.528      0.000      -1.026      -0.874
==============================================================================
Omnibus:                      137.043   Durbin-Watson:                   0.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.373
Skew:                           1.453   Prob(JB):                     5.36e-64
Kurtosis:                       5.319   Cond. No.                         29.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Predictions

In [6]:
new_df = pd.DataFrame({'lstat':[5, 10, 15]})
variables = ['lstat']
newX = transform_df_for_model(new_df, variables)
newX

,intercept,lstat
0,1.0,5
1,1.0,10
2,1.0,15


In [7]:
new_predictions = results.get_prediction(newX)
print(new_predictions.predicted_mean)
new_predictions.conf_int(alpha=0.05)

[29.80359411 25.05334734 20.30310057]


array([[29.00741194, 30.59977628],
       [24.47413202, 25.63256267],
       [19.73158815, 20.87461299]])

Prediction interval

In [8]:
new_predictions.conf_int(obs=True, alpha=0.05)

array([[17.56567478, 42.04151344],
       [12.82762635, 37.27906833],
       [ 8.0777421 , 32.52845905]])

# Linear regression : 2 variables

In [9]:
Boston = pd.read_csv("data/Boston.csv")
X = transform_df_for_model(Boston,['lstat', 'age'])
model1 = sm.OLS(y, X)
results1 = model1.fit()
summarize(results1)

,coef,std err,t,P>|t|
intercept,33.2228,0.731,45.458,0.000
lstat,-1.0321,0.048,-21.416,0.000
age,0.0345,0.012,2.826,0.005


In [10]:
terms = Boston.columns.drop("medv")
X = transform_df_for_model(Boston, terms)
model = sm.OLS(y, X)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,41.6173,4.936,8.431,0.000
crim,-0.1214,0.033,-3.678,0.000
zn,0.0470,0.014,3.384,0.001
indus,0.0135,0.062,0.217,0.829
chas,2.8400,0.870,3.264,0.001
nox,-18.7580,3.851,-4.870,0.000
rm,3.6581,0.420,8.705,0.000
age,0.0036,0.013,0.271,0.787
dis,-1.4908,0.202,-7.394,0.000
rad,0.2894,0.067,4.325,0.000


## with interactions

In [11]:
X = transform_df_for_model(Boston, ['lstat','age'],interactions = [('lstat', 'age')])
model2 = sm.OLS(y, X)

results2 = model2.fit()
summarize(model2.fit())
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     209.3
Date:                Tue, 18 Mar 2025   Prob (F-statistic):           4.86e-88
Time:                        21:49:56   Log-Likelihood:                -1635.0
No. Observations:                 506   AIC:                             3278.
Df Residuals:                     502   BIC:                             3295.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     36.0885      1.470     24.553      0.000      33.201      38.976
lstat         -1.3921      0.167     -8.313      0.000      -1.721      -1.063
age           -0.0007      0.020     -0.036      0.971      -0.040       0.038
lstat:age      0.0042      0.002      2.244      0.025       0.001       0.008
==============================================================================
Omnibus:                      135.601   Durbin-Watson:                   0.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              296.955
Skew:                           1.417   Prob(JB):                     3.29e-65
Kurtosis:                       5.461   Cond. No.                     6.88e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.88e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

How to use anova_lm
anova(reduced,full)
Ho no need the full model


In [12]:
anova_lm(results1,results2)

,df_resid,ssr,df_diff,ss_diff,F,Pr(>F)
0,503.0,19168.128609,0.0,NaN,NaN,NaN
1,502.0,18977.716145,1.0,190.412464,5.036805,0.025249


In [13]:
Carseats = pd.read_csv("data/Carseats.csv")

allvars = list(Carseats.columns.drop('Sales'))
y = Carseats['Sales']

X = transform_df_for_model(Carseats, allvars,[('Income', 'Advertising'),
                   ('Price', 'Age')])
model = sm.OLS(y, X)
summarize(model.fit())

,coef,std err,t,P>|t|
intercept,6.5756,1.009,6.519,0.000
CompPrice,0.0929,0.004,22.567,0.000
Income,0.0109,0.003,4.183,0.000
Advertising,0.0702,0.023,3.107,0.002
Population,0.0002,0.000,0.433,0.665
Price,-0.1008,0.007,-13.549,0.000
ShelveLoc[Good],4.8487,0.153,31.724,0.000
ShelveLoc[Medium],1.9533,0.126,15.531,0.000
Age,-0.0579,0.016,-3.633,0.000
Education,-0.0209,0.020,-1.063,0.288


# Split


In [14]:
Auto = pd.read_csv('data/Auto.csv')
Auto_train, Auto_test = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)

## use of partial

In [15]:
hp_mm = partial(transform_df_for_model, terms = ['horsepower'])
X_train = hp_mm(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

In [16]:
X_test = hp_mm(Auto_test)
y_test = Auto_test['mpg']
test_pred = results.predict(X_test)
np.mean((y_test - test_pred)**2).item()

23.616617069669893

## Function to compute MSE

In [17]:
def evalMSE(terms,
            response,
            train,
            test):

   mm_transform = partial(transform_df_for_model, terms = terms)
   X_train = mm_transform(train)
   y_train = train[response]

   X_test = mm_transform(test)
   y_test = test[response]

   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)

   return np.mean((y_test - test_pred)**2).item()

In [18]:
evalMSE(terms =['horsepower','cylinders'],response ='mpg',train = Auto_train, test=Auto_test)

20.15359485880563

In [20]:
Auto = pd.read_csv("data/Auto.csv")
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
hp_model = make_pipeline(
    CustomDataTransformer(['horsepower']),
    sklearn_sm(sm.OLS))

X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

np.float64(24.231513517929226)

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold
import pandas as pd

# 📌 Charger les données
Auto = pd.read_csv("data/Auto.csv")
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']

# 📌 Variables pour la transformation
terms = ['horsepower']

# 📌 Définir le pipeline
pipeline = make_pipeline(
    CustomDataTransformer(terms),
sklearn_sm(sm.OLS))

# 📌 Validation croisée
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, Y, cv=cv, scoring="neg_mean_squared_error")  # MSE négatif

# 📌 Affichage des résultats
print("Scores de validation croisée (MSE négatif) :", scores)
print("MSE moyen :", -np.mean(scores))  # Convertir en MSE positif



Scores de validation croisée (MSE négatif) : [-22.15323712 -29.14787841 -26.67385214 -19.52413152 -23.60002108]
MSE moyen : 24.219824054860855


In [22]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.4244303 , 19.03321527])

In [23]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848402, 19.13718373])

In [24]:
# 📌 Définir le pipeline
hp_model = make_pipeline(
    CustomDataTransformer(terms),
sklearn_sm(sm.OLS))

In [25]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score']

array([23.61661707])